## Predictor Bayesiano

Para el laboratorio 2 de Aprendizaje Automático, se pide implementar un predictor bayesiano de palabras, como los que se usa en los motores de búsqueda o teclados de los teléfonos. Dadas las N palabras previas (hiperparámetro del predictor), se deberá recomendar la siguiente palabra, habiendo entrenado el sistema con conversaciones recolectadas de grupos de Whatsapp.



### Recolección y pre procesamiento de datos

Se descargaron las conversaciones de un grupo de Whatsapp con al rededor de 43000 mensajes. Estas quedaron disponibles desde la aplicación en texto plano (.txt) y fueron cargadas en pandas, descartando los campos de autor y fecha del mensaje. Cabe destacar que los archivos proporcionados por IOS y Android tienen leves diferencias en formato, por lo que el código de carga diferencia entre los dos. Luego se pasó a eliminar encabezados, emoticones, mensajes que indicaban que en ese punto de la conversación habia contenido multimedia y números




In [1]:
from preprocess import load_wpp_data
WPP_FILENAME='Datos/chat_big.txt'

data=load_wpp_data(WPP_FILENAME)


Es IOS?:  False


#### Diccionario

Para mejorar el desempeño del predictor (tanto en tiempo como en su predicción)  fue necesario limitar el vocabulario admitido. Se incluye un diccionario de español, que se utiliza para el filtrado.

In [2]:
SPANISH_DICT_FILENAME='Datos/es.txt'
with open(SPANISH_DICT_FILENAME, 'r', encoding='utf-8') as archivo:
    palabras_validas=set()
    for linea in archivo:
        palabra = linea.strip()
        palabras_validas.add(palabra)

### Algoritmo

Dadas N palabras, se debe predecir la siguiente. Para ejemplificar, tomemos 4 palabras previas u horizonte. Si el usuario por ejemplo escribió "feliz cumple que pases" el sistema debería recomendar "lindo". Esperamos entonces que sea "lindo" el valor donde se maximice 

$$p(hipotesis)*p(feliz/hipotesis)*p(feliz/hipotesis)*p(feliz/hipotesis)*p(feliz/hipotesis)$$

para todas las opciones de nuestro vocabulario.

Con el fin de mejorar el predictor ante casos en los que p(horizonte/hipotesis) sea cero, se introduce el m_estimador visto en el curso. Este busca crear muestras ficticias, tantas como se especifique en el parametro m.

m_estimador=$$\frac{e+m*p}{(n+m)}$$

siendo e la cantidad de ejemplos que reafirman hipotesis con una palabra horizonte, m la cantidad de muestras que queremos "agregar",
n el total de ejemplos de la hipotesis y p 1/|vocabulario|



### Implementación

Se utilizaron dos diccionarios para contener el conjunto de datos. 
El primer diccionario, "priori" contiene la frecuencia de cada palabra
El segundo, "posteriori" tiene para cada palabra del vocabulario, la frecuencia de las palabras que estuvieron en su horizonte

Estos dos primeros diccionarios se utilizan para generar el tercero que contiene los estimadores para cada combinacion de palabras en "posteriori" y un campo extra para la probabilidad de palabras que no se hayan visto junto a la hipotesis antes (_default, o P_nada en el curso)

La ventaja de almacenar estos dos diccionarios intermedios es que, para actualizar los datos luego de cada frase completa, solo es necesario actualizar los m-estimadores de las palabras hipotesis relacionadas, agilizando la actualización

## Cliente
Inicializamos la clase del estimador, estableciendo los hiperparametros.

In [3]:
from bayes import BayesPredictor
HORIZONTE=4
m=2
predictor=BayesPredictor(data["palabras"],HORIZONTE,m,palabras_validas)
print(predictor.vocab())

entrenando todo
['hay', 'como', 'muy', 'buenas', 'igualmente', 'las', 'saco', 'de', 'todo', 'el', 'coro', 'si', 'me', 'gustaron', 'cada', 'vez', 'la', 'veo', 'le', 'pido', 'nos', 'saque', 'fotos', 'que', 'son', 'esos', 'primeros', 'planos', 'espantosos', 'por', 'dios', 'divina', 'saliste', 'te', 'en', 'una', 'linda', 'yo', 'no', 'tan', 'bien', 'comparti', 'lindas', 'pera', 'amiga', 'encima', 'descansas', 'se', 'hacer', 'eso', 'juicio', 'otras', 'un', 'poco', 'turbias', 'vero', 'escribe', 'lo', 'piensa', 'ah', 'estabas', 'pidiendo', 'mi', 'comparta', 'bue', 'tienen', 'desde', 'los', 'encargados', 'medios', 'cuesta', 'responder', 'mensajes', 'grupo', 'alguien', 'calculen', 'meter', 'ya', 'es', 'too', 'ves', 'esas', 'vos', 'tipo', 'esa', 'felipe', 'preciosa', 'dos', 'primer', 'plano', 'hagas', 'tal', 'cual', 'dije', 'saliera', 'fea', 'odio', 'gente', 'hace', 'cosas', 'digo', 'serio', 'queres', 'hablemos', 'tu', 'cara', 'mejor', 'chau', 'pero', 'empiezan', 'ustedes', 'calientes', 'mal', 'n

In [4]:


def recomendacion_bayesiana(frase):
  

  return predictor.predict(frase)



##### LOOP PRINCIPAL #####

print("Ingrese la frase dando ENTER luego de \x1b[3mcada palabra\x1b[0m.")
print("Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER")
print("Ingrese '.' para comenzar con una frase nueva.")
print("Ingrese '..' para terminar el proceso.")

frase = []
palabra_sugerida = ""
while 1:
    palabra = input(">> ")

    if palabra == "..":
        break

    elif palabra == ".":
        predictor.update(frase,solo_cambios=True)
            
        print("----- Comenzando frase nueva -----")
        frase = []

    elif palabra == "": # acepta última palabra sugerida
        frase.append(palabra_sugerida)

    else: # escribió una palabra
        frase.append(palabra)

    if frase:
        palabra_sugerida = recomendacion_bayesiana(frase)
    
        frase_propuesta = frase.copy()
        frase_propuesta.append("\x1b[3m"+ palabra_sugerida +"\x1b[0m")
    
        print(" ".join(frase_propuesta))



Ingrese la frase dando ENTER luego de cada palabra.
Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER
Ingrese '.' para comenzar con una frase nueva.
Ingrese '..' para terminar el proceso.


feliz cumple
feliz cumple pases
feliz cumple pases lindo
feliz cumple pases lindo beso
feliz cumple pases lindo beso grande
feliz cumple pases lindo beso grande grande
feliz cumple pases lindo beso grande grande lamento
